In [11]:
from flask import Flask, request, jsonify
import cv2
import mediapipe as mp
import pyautogui
import util
import random
from pynput.mouse import Button, Controller

In [12]:
# Initialize components
app = Flask(__name__)
mouse = Controller()
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

screen_width, screen_height = pyautogui.size()

In [13]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

    return None

In [14]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width) # Getting actual screen height & width
        y = int(index_finger_tip.y * screen_height)

        pyautogui.moveTo(x, y)

In [15]:
def is_left_click(landmarks_list, thumb_index_dist):
    return (
            util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 90 and
            thumb_index_dist > 50
    )

def is_right_click(landmarks_list, thumb_index_dist):
    return (
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and
            util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90  and
            thumb_index_dist > 50
    )

def is_double_click(landmarks_list, thumb_index_dist):
    return (
            util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and
            thumb_index_dist > 50
    )

def is_screenshot(landmarks_list, thumb_index_dist):
    return (
            util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and
            thumb_index_dist < 50
    )


In [16]:
def detect_gestures(landmarks_list, processed):
    results = {}
    if len(landmarks_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmarks_list[4], landmarks_list[5]])
        index_angle = util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8])

        if thumb_index_dist < 50 and index_angle > 90:
            move_mouse(index_finger_tip)
            results["action"] = "mouse_movement"
        elif is_left_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            results["action"] = "left_click"
        elif is_right_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            results["action"] = "right_click"
        elif is_double_click(landmarks_list, thumb_index_dist):
            pyautogui.doubleClick()
            results["action"] = "double_click"
        elif is_screenshot(landmarks_list, thumb_index_dist):
            label = random.randint(1, 1000)
            pyautogui.screenshot(f'my_screenshot_{label}.png')
            results["action"] = "screenshot"
    return results

In [18]:
@app.route('/process_frame', methods=['POST'])
def process_frame():
    try:
        # Parse frame input from request
        image = request.files['frame'].read()
        np_arr = np.frombuffer(image, np.uint8)
        frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        processed = hands.process(frame_rgb)

        landmarks_list = []
        if processed.multi_hand_landmarks:
            hand_landmarks = processed.multi_hand_landmarks[0]
            for lm in hand_landmarks.landmark:
                landmarks_list.append((lm.x, lm.y))

        results = detect_gestures(landmarks_list, processed)
        return jsonify({"success": True, "results": results})
    except Exception as e:
        return jsonify({"success": False, "error": str(e)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

AssertionError: View function mapping is overwriting an existing endpoint function: process_frame